In [229]:
import pandas as pd
import numpy as np
# Ruta del archivo CSV
ruta_csv = r'C:\\Users\\USER\OneDrive\\Escritorio\\Maestría\\mi_proyecto_ml\\src\\House_Rent_Dataset.csv'

# Leer el archivo CSV
try:
    data = pd.read_csv(ruta_csv)
    # Mostrar las primeras filas del dataset
    print(data.head())
except FileNotFoundError:
    print("El archivo no se encuentra en la ruta especificada. Por favor, verifica la ruta y el nombre del archivo.")

In [230]:
data

In [231]:
data.info()

In [232]:
# Información general del dataset
#print(data.info())

# Estadísticas descriptivas
#print(data.describe())

# Visualización de algunas características
import matplotlib.pyplot as plt
import seaborn as sns

# Distribución del precio de alquiler
plt.figure(figsize=(10, 6))
sns.histplot(data['Rent'], bins=30, kde=True)
plt.title('Distribución del Precio de Alquiler')
plt.xlabel('Precio de Alquiler')
plt.ylabel('Frecuencia')
plt.show()

In [233]:
data.shape

In [234]:
# Manejo de valores nulos
data = data.dropna()
data = data.drop(columns=['Posted On'])


# Quitar el 2% superior e inferior en relación con la variable 'Rent'
lower_bound = data['Rent'].quantile(0.02)
upper_bound = data['Rent'].quantile(0.98)

data_filtered = data[(data['Rent'] > lower_bound) & (data['Rent'] < upper_bound)]

In [ ]:
from sklearn.preprocessing import LabelEncoder
# Identificar columnas categóricas automáticamente
categorical_cols = data.select_dtypes(include=['object']).columns
# Aplicar Label Encoding a cada columna categórica
encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = encoder.fit_transform(data[col])
print("Columnas transformadas:", categorical_cols)


In [236]:
data['Rent'] = np.log1p(data['Rent'])
data

In [237]:
X = data.drop(columns=['Rent'])
y = data['Rent'] 

In [238]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [239]:
from sklearn.model_selection import train_test_split

# 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 80% de entrenamiento se divide en entrenamiento (80%) y validación (20%)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [240]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([
    Dense(512, activation='relu', kernel_regularizer=l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dense(1, activation='linear')
])

###fdf"

In [242]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

In [243]:
history = model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_val, y_val))

In [244]:
loss, mae = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, MAE: {mae}')

In [245]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Actualización 11:07